In [44]:
import pandas as pd
import pydeck

In [45]:
# Data Imports
airlines = pd.read_csv('./data/airlines.csv')
airplanes = pd.read_csv('./data/airplanes.csv')
airports = pd.read_csv('./data/airports.csv')
routes = pd.read_csv('./data/routes.csv')

In [46]:
airports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7698 entries, 0 to 7697
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Airport ID             7698 non-null   int64  
 1   Name                   7698 non-null   object 
 2   City                   7649 non-null   object 
 3   Country                7698 non-null   object 
 4   IATA                   7698 non-null   object 
 5   ICAO                   7698 non-null   object 
 6   Latitude               7698 non-null   float64
 7   Longitude              7698 non-null   float64
 8   Altitude               7698 non-null   int64  
 9   Timezone               7698 non-null   object 
 10  DST                    7698 non-null   object 
 11  Tz database time zone  7698 non-null   object 
 12  Type                   7698 non-null   object 
 13  Source                 7698 non-null   object 
dtypes: float64(2), int64(2), object(10)
memory usage: 842.1+

In [47]:
routes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67663 entries, 0 to 67662
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Airline                 67663 non-null  object
 1   Airline ID              67663 non-null  object
 2   Source airport          67663 non-null  object
 3   Source airport ID       67663 non-null  object
 4   Destination airport     67663 non-null  object
 5   Destination airport ID  67663 non-null  object
 6   Codeshare               14597 non-null  object
 7   Stops                   67663 non-null  int64 
 8   Equipment               67645 non-null  object
dtypes: int64(1), object(8)
memory usage: 4.6+ MB


In [48]:
valid_routes = routes[routes['Destination airport ID'].str.isnumeric() & routes['Source airport ID'].str.isnumeric()]
valid_routes[['Source airport ID', 'Destination airport ID']] = valid_routes[['Source airport ID', 'Destination airport ID']].astype(int)

/Users/willsmith/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [49]:
src_locs = airports[['Name', 'IATA', 'City', 'Country', 'Latitude', 'Longitude']].copy()
dest_locs = src_locs.copy()
# src_locs = src_locs.rename(columns={'Latitude': 'src_lat', 'Longitude': 'src_long'})
# dest_locs = dest_locs.rename(columns={'Latitude': 'dest_lat', 'Longitude': 'dest_long'})
src_locs = src_locs.add_prefix('src_')
dest_locs = dest_locs.add_prefix('dest_')

In [50]:
full = pd.merge(routes, src_locs, left_on='Source airport', right_on='src_IATA', how='left')
full = pd.merge(full, dest_locs, left_on='Destination airport', right_on='dest_IATA', how='left')

In [51]:
# Export dataset to file
full.to_csv('routes.csv',index=False)

In [52]:
airlines

,Airline ID,Name,Alias,IATA,ICAO,Callsign,Country,Active
0,-1,Unknown,\N,-,NaN,\N,\N,Y
1,1,Private flight,\N,-,NaN,NaN,NaN,Y
2,2,135 Airways,\N,NaN,GNL,GENERAL,United States,N
3,3,1Time Airline,\N,1T,RNX,NEXTIME,South Africa,Y
4,4,2 Sqn No 1 Elementary Flying Training School,\N,NaN,WYT,NaN,United Kingdom,N
...,...,...,...,...,...,...,...,...
6157,21248,GX Airlines,NaN,NaN,CBG,SPRAY,China,Y
6158,21251,Lynx Aviation (L3/SSX),NaN,NaN,SSX,Shasta,United States,N
6159,21268,Jetgo Australia,NaN,JG,\N,NaN,Australia,Y
6160,21270,Air Carnival,NaN,2S,\N,NaN,India,Y


In [53]:
coords = full[['src_Latitude', 'src_Longitude', 'dest_Latitude', 'dest_Longitude']]

In [54]:
coords_valid = coords[~coords.isna().any(axis=1)]

In [55]:
coords_valid.dtypes
coords_valid.to_csv('valid_coords.csv',index=False)

In [56]:
full = full[~full[['src_Latitude', 'src_Longitude', 'dest_Latitude', 'dest_Longitude']].isna().any(axis=1)]

In [57]:
full = full[~full.isna().any(axis=1)]

In [58]:
full.isna().any(axis=0)
full.to_csv('full_routes_data.csv',index=False)

In [59]:
full.loc[(full.src_Country == "United States") & (full.dest_Country == "United States")]

,Airline,Airline ID,Source airport,Source airport ID,Destination airport,Destination airport ID,Codeshare,Stops,Equipment,src_Name,...,src_City,src_Country,src_Latitude,src_Longitude,dest_Name,dest_IATA,dest_City,dest_Country,dest_Latitude,dest_Longitude
2547,7H,16726,ABL,7177,OTZ,3693,Y,0,CNC,Ambler Airport,...,Ambler,United States,67.106300,-157.856989,Ralph Wien Memorial Airport,OTZ,Kotzebue,United States,66.884697,-162.598999
2548,7H,16726,ABL,7177,SHG,7117,Y,0,CNC,Ambler Airport,...,Ambler,United States,67.106300,-157.856989,Shungnak Airport,SHG,Shungnak,United States,66.888100,-157.162003
2550,7H,16726,AIN,7220,ATK,6714,Y,0,CNC,Wainwright Airport,...,Wainwright,United States,70.638000,-159.994995,Atqasuk Edward Burnell Sr Memorial Airport,ATK,Atqasuk,United States,70.467300,-157.436005
2551,7H,16726,AIN,7220,BRW,3571,Y,0,BE1 CNC,Wainwright Airport,...,Wainwright,United States,70.638000,-159.994995,Wiley Post Will Rogers Memorial Airport,BRW,Barrow,United States,71.285402,-156.766008
2565,7H,16726,ANI,5967,CHU,7091,Y,0,CNA,Aniak Airport,...,Aniak,United States,61.581600,-159.542999,Chuathbaluk Airport,CHU,Chuathbaluk,United States,61.579102,-159.216003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64611,WS,5416,LGA,3697,ATL,3682,Y,0,M88 757 320 738 739 73W,La Guardia Airport,...,New York,United States,40.777199,-73.872597,Hartsfield Jackson Atlanta International Airport,ATL,Atlanta,United States,33.636700,-84.428101
64612,WS,5416,LGA,3697,MSY,3861,Y,0,319 CR9,La Guardia Airport,...,New York,United States,40.777199,-73.872597,Louis Armstrong New Orleans International Airport,MSY,New Orleans,United States,29.993401,-90.258003
64625,WS,5416,MIA,3576,MSY,3861,Y,0,738,Miami International Airport,...,Miami,United States,25.793200,-80.290604,Louis Armstrong New Orleans International Airport,MSY,New Orleans,United States,29.993401,-90.258003
64630,WS,5416,MSY,3861,LGA,3697,Y,0,319 CR9,Louis Armstrong New Orleans International Airport,...,New Orleans,United States,29.993401,-90.258003,La Guardia Airport,LGA,New York,United States,40.777199,-73.872597


In [62]:
airlines

,Airline ID,Name,Alias,IATA,ICAO,Callsign,Country,Active
0,-1,Unknown,\N,-,NaN,\N,\N,Y
1,1,Private flight,\N,-,NaN,NaN,NaN,Y
2,2,135 Airways,\N,NaN,GNL,GENERAL,United States,N
3,3,1Time Airline,\N,1T,RNX,NEXTIME,South Africa,Y
4,4,2 Sqn No 1 Elementary Flying Training School,\N,NaN,WYT,NaN,United Kingdom,N
...,...,...,...,...,...,...,...,...
6157,21248,GX Airlines,NaN,NaN,CBG,SPRAY,China,Y
6158,21251,Lynx Aviation (L3/SSX),NaN,NaN,SSX,Shasta,United States,N
6159,21268,Jetgo Australia,NaN,JG,\N,NaN,Australia,Y
6160,21270,Air Carnival,NaN,2S,\N,NaN,India,Y


In [64]:
full = full[full['Airline ID'].str.isnumeric()]
full['Airline ID'] = full['Airline ID'].astype(int)
full = pd.merge(full,airlines,how="left",on="Airline ID")

In [65]:
full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14475 entries, 0 to 14474
Data columns (total 28 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Airline                 14475 non-null  object 
 1   Airline ID              14475 non-null  int64  
 2   Source airport          14475 non-null  object 
 3   Source airport ID       14475 non-null  object 
 4   Destination airport     14475 non-null  object 
 5   Destination airport ID  14475 non-null  object 
 6   Codeshare               14475 non-null  object 
 7   Stops                   14475 non-null  int64  
 8   Equipment               14475 non-null  object 
 9   src_Name                14475 non-null  object 
 10  src_IATA                14475 non-null  object 
 11  src_City                14475 non-null  object 
 12  src_Country             14475 non-null  object 
 13  src_Latitude            14475 non-null  float64
 14  src_Longitude           14475 non-null

In [66]:
full

,Airline,Airline ID,Source airport,Source airport ID,Destination airport,Destination airport ID,Codeshare,Stops,Equipment,src_Name,...,dest_Country,dest_Latitude,dest_Longitude,Name,Alias,IATA,ICAO,Callsign,Country,Active
0,2P,897,GES,2402,MNL,2397,Y,0,320,General Santos International Airport,...,Philippines,14.508600,121.019997,Air Philippines,\N,2P,GAP,ORIENT PACIFIC,Philippines,Y
1,2P,897,MNL,2397,GES,2402,Y,0,320,Ninoy Aquino International Airport,...,Philippines,6.058000,125.096001,Air Philippines,\N,2P,GAP,ORIENT PACIFIC,Philippines,Y
2,4M,3201,DFW,3670,EZE,3988,Y,0,777,Dallas Fort Worth International Airport,...,Argentina,-34.822200,-58.535800,LAN Argentina,\N,4M,DSM,LAN AR,Argentina,Y
3,4M,3201,EZE,3988,DFW,3670,Y,0,777,Ministro Pistarini International Airport,...,United States,32.896801,-97.038002,LAN Argentina,\N,4M,DSM,LAN AR,Argentina,Y
4,4M,3201,EZE,3988,JFK,3797,Y,0,777,Ministro Pistarini International Airport,...,United States,40.639801,-73.778900,LAN Argentina,\N,4M,DSM,LAN AR,Argentina,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14470,ZH,4611,ZHA,6357,PEK,3364,Y,0,737,Zhanjiang Airport,...,China,40.080101,116.584999,Shenzhen Airlines,\N,ZH,CSZ,SHENZHEN AIR,China,Y
14471,ZH,4611,ZUH,6355,CKG,3393,Y,0,737,Zhuhai Jinwan Airport,...,China,29.719200,106.641998,Shenzhen Airlines,\N,ZH,CSZ,SHENZHEN AIR,China,Y
14472,ZH,4611,ZUH,6355,CTU,3395,Y,0,319,Zhuhai Jinwan Airport,...,China,30.578501,103.946999,Shenzhen Airlines,\N,ZH,CSZ,SHENZHEN AIR,China,Y
14473,ZH,4611,ZUH,6355,PEK,3364,Y,0,737 320,Zhuhai Jinwan Airport,...,China,40.080101,116.584999,Shenzhen Airlines,\N,ZH,CSZ,SHENZHEN AIR,China,Y


In [67]:
full.Name.unique()

array(['Air Philippines', 'LAN Argentina', 'Aeroflot-Nord',
       'Canadian North', 'First Air', 'Era Alaska', 'Cape Air',
       'Jet Airways', 'Aegean Airlines', 'American Airlines',
       'Air Berlin', 'Air Canada', 'Azul', 'Mandarin Airlines',
       'Air France', 'Air India Limited', 'AeroMéxico', 'Air One',
       'Aerolineas Argentinas', 'Alaska Airlines', 'Royal Air Maroc',
       'Avianca - Aerovias Nacionales de Colombia', 'Finnair', 'Alitalia',
       'Belavia Belarusian Airlines', 'Uni Air', 'British Airways',
       'Flybe', 'EVA Air', 'Air Baltic', 'Air China', 'China Airlines',
       'Copa Airlines', 'Cubana de Aviación', 'Cathay Pacific',
       'Cyprus Airways', 'China Southern Airlines', 'Golden Air',
       'Condor Flugdienst', 'Delta Air Lines', 'Norwegian Air Shuttle',
       'Aer Lingus', 'Emirates', 'Ethiopian Airlines', 'Etihad Airways',
       'Flybaboo', 'Frontier Airlines', 'Air Pacific', 'AirTran Airways',
       'Shanghai Airlines', 'Garuda Indonesia', '

In [68]:
full.to_csv('./full_routes_data.csv',index=False)